<a href="https://colab.research.google.com/github/partizanos/advanced_image_processing/blob/master/AEI_tp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tp4 image processing JPEG -- DIMITRIS PROIOS

## Exercise 1 (a) Implement by yourself the JPEG compression algorithm described above.
### 1.Color space transformation :
#### RGB color space to YCbCr color space conversion:



First, the image should be converted from RGB into a different color space called Y′CBCR (or, informally, YCbCr). 

It has three components: 
- the Y' component represents the brightness of a pixel, 
- CB and CR components represent the chrominance (split into blue and red components). 

The Y′CBCR color space conversion allows greater compression without a significant effect on perceptual image quality (or greater perceptual image quality for the same compression).

In [0]:

def extract_colors(img):
  R_DIMENSION = img[:, :, 0]
  G_DIMENSION = img[:, :, 1]
  B_DIMENSION = img[:, :, 2]
  return R_DIMENSION, G_DIMENSION, B_DIMENSION



# original implementation of descrition

def YCbCr(R_DIMENSION,G_DIMENSION,B_DIMENSION): 
  
  Y = np.add(np.add(0.299 * R_DIMENSION, 0.587 * G_DIMENSION), 0.114 * B_DIMENSION)
  
  Cb = - 0.1687 * R_DIMENSION - 0.3313 * G_DIMENSION + 0.5 * B_DIMENSION + 128
  
  Cr = 0.5 * R_DIMENSION - 0.4187 * G_DIMENSION - 0.0813 * B_DIMENSION + 128
  
  R_DIMENSION = Y + 1.402 * (Cr - 128)
  
  G_DIMENSION = Y - 0.34414 * (Cb - 128) - 0.71414 * (Cr - 128)
  
  B_DIMENSION = Y + 1.772 * (Cb - 128)
  
  return Y, Cb, Cr, R_DIMENSION, G_DIMENSION, B_DIMENSION



### 2.  Downsampling 

#### Keep the Y component. and downsample the Cb and Cr components in 2 times.


- Due to color and brightness densities, humans can see considerably more fine detail in the brightness of an image (the Y' component)  than in the hue and color saturation of an image (the Cb and Cr components). Using this knowledge, encoders can be designed to compress images more efficiently.


- The transformation into the Y′CBCR color model enables reduce the spatial resolution of the Cb and Cr components (called "downsampling" or "chroma subsampling"). The ratios at which the downsampling is ordinarily done for JPEG images are: 
  - 4:4:4 (no downsampling)
  - 4:2:2 (reduction by a factor of 2 in the horizontal direction), 
  - (most commonly) 4:2:0 (reduction by a factor of 2 in both the horizontal and vertical directions). 

For the rest of the compression process, Y', Cb and Cr are processed separately and in a very similar manner.

In [0]:
# TODO try different downsample method https://stackoverflow.com/questions/13242382/resampling-a-numpy-array-representing-an-image
# verify matlab downsampl

def downsampleChannels(Cb, Cr, dimensions_number=2):
  def downsample_image(img, skip):
      return img[::skip,::skip]
  
  # imresize 
  d_cb  = downsample_image(Cb, 2)
  #   d_cb  = downsample_image(d_cb, 2)
  d_cr  = downsample_image(Cr, 2)
  #   d_cr  = downsample_image(d_cr, 2)
  return d_cb, d_cr

In [3]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import math
from scipy import fftpack 


def main():
  response = requests.get("https://raw.githubusercontent.com/partizanos/advanced_image_processing/master/TP/TP4/test.png")
  im = Image.open(BytesIO(response.content))
  img = np.array(im)
  M, N, _ = img.shape
  max_M = int(math.modf(M/8)[1] *8)
  max_N = int(math.modf(N/8)[1] *8)
  crop = lambda img:img[:max_M, :max_N, :]
  img = crop(img )
  return img, M, N,max_M, max_N



def test(showImg=False):
  img, M, N,max_M, max_N = main()
  if(showImg):
    plt.imshow(img, cmap=plt.cm.Spectral)
    plt.title("Orignal")
  
  
  R, G, B = extract_colors(img)
  
  
  assert R.shape == (864, 1296)
  assert G.shape == (864, 1296)
  assert B.shape == (864, 1296)
  
  Y, Cb, Cr, R, G, B  = YCbCr(R, G, B )
  
  assert R.shape == (864, 1296)
  assert G.shape == (864, 1296)
  assert B.shape == (864, 1296)
  assert Y.shape == (864, 1296)
  assert Cr.shape == (864, 1296)
  assert Cb.shape == (864, 1296)
  
  
  d_cb, d_cr = downsampleChannels(Cb, Cr)
  assert Y.shape == (864, 1296)
  assert d_cb.shape == (432, 648)

  return(
      img, M, N,max_M, max_N , R, G, B , Y, Cb, Cr, d_cb, d_cr 
  )




img, M, N,max_M, max_N , R, G, B , Y, Cb, Cr, d_cb, d_cr  = test()
print("test succeed")

test succeed


### 3. Preprocessing for DCT transformation

#### 3.1 Split the image into 8 × 8 non-overlapping blocks

In [4]:
def img_2_block(Y, cb, cr, number_octads):
  number_octads = int(max_M* max_N / 64)
  Y_r = Y.reshape((number_octads, 8,8))
  Cb_r = Cb.reshape((number_octads, 8,8))
  Cr_r = Cr.reshape((number_octads, 8,8))
  return Y_r, Cb_r,Cr_r
  
# test TODO grayscale image 
#test
#   Y_r[0] = [
#       [52, 55, 61, 66, 70, 61, 64, 73],
#       [63, 59, 66, 90, 109, 85, 69, 72],
#       [62, 59, 68, 113, 144, 104, 66, 73],
#       [63, 58, 71, 122, 154, 106, 70, 69],
#       [67, 61, 68, 104, 126, 88, 68, 70],
#       [79, 65, 60, 70, 77, 63, 58, 75],
#       [85, 71, 64, 59, 55, 61, 65, 83],
#       [87, 79, 69, 68, 65, 76, 78, 94]
#   ]





Y_r, Cb_r, Cr_r = img_2_block(Y, d_cb, d_cr, (max_M, max_N ) )

# TODO test
Y_r.shape, Cb_r.shape, Cr_r.shape


((17496, 8, 8), (17496, 8, 8), (17496, 8, 8))

#### 3.2 In each block subtract global mean computed as 2 k−1 , where k is the number of gray levels in the image

In [5]:
def global_mean(img):
  global_mean = 128
  img=img - global_mean
  img = img.astype("int")
  return img

Y_r = global_mean(Y_r)
Cb_r = global_mean(Cb_r)
Cr_r = global_mean(Cr_r)
Y_r.shape, Y_r[0]

((17496, 8, 8), array([[-110, -108, -101, -114, -115, -112, -114, -117],
        [-116, -114, -113, -107, -105, -104,  -98,  -94],
        [ -86,  -90,  -98, -105, -110,  -99, -103, -106],
        [-110, -109, -102, -101,  -91,  -73,  -73,  -63],
        [ -35,  -27,  -77, -102,  -90,  -67,  -64,  -70],
        [ -66,  -94,  -97,  -95,  -97,  -94,  -86,  -41],
        [  13,   26,   30,   14,  -10,  -22,   22,   31],
        [  40,   33,    0,  -38,  -44,   11,    5,  -28]]))

### Step 4. DCT transformation per block: T (u, v)

In [6]:
# TODO aply and inverse image should be same MSE =10** -10
def dct_2d(image):
  return fftpack.dct(fftpack.dct(image.T, norm='ortho').T, norm='ortho')

  
def ict_2d(): 
  return fftpack.dct(fftpack.dct(image.T, norm='ortho').T, norm='ortho')

Y_dct = dct_2d(Y_r)
Cb_dct = dct_2d(Cb_r)
Cr_dct = dct_2d(Cr_r)
Y_dct[0]


array([[-2.32424058e+04,  1.48205303e+01,  1.61819283e+01,
        -1.56592549e+01, -2.44144322e+01, -6.41182556e+00,
        -1.04407181e+01, -2.21986564e+00],
       [-2.31538227e+04, -4.99256123e+01, -2.61065820e+01,
         6.83830692e+00,  5.28970455e+00, -1.46695910e+01,
         5.76516143e+00,  6.54256376e+00],
       [-2.32621961e+04,  1.75776026e+00,  5.30119326e+00,
        -8.99781826e+00, -3.43202660e+00,  5.68003455e-01,
        -3.02905205e+00,  1.42639779e+00],
       [-2.31532560e+04, -1.55695129e+01, -3.25400242e+01,
        -4.37613444e-01, -3.07652852e+00, -4.12470487e+00,
        -8.42548523e+00, -9.65467050e-01],
       [-2.32600551e+04, -1.88533151e+01,  1.49542199e+01,
         3.14233930e+00,  2.84665758e+00, -6.67631632e-01,
        -5.32340389e+00,  7.48799404e-01],
       [-2.31335406e+04, -2.72705147e+00, -2.90658725e+01,
        -4.60730132e+00, -1.04243799e-01, -1.35655872e+00,
         7.30172826e-01,  2.95232948e+00],
       [-2.32723585e+04,  1.497279

### Step 5. Block coefficients quantization:

$T(u,v) = round \frac{T(u,v)} {Z(u,v)}  $

In [8]:
def quantization(Y_dct, Cb_dct, Cr_dct, Z):
  T_Y = np.round(Y_dct/Z)
  T_cb = np.round(Cb_dct/Z)
  T_cr = np.round(Cr_dct/Z)
  
  return (T_Y, T_cb, T_cb)
 
  
######################


Z = [
      [16, 11, 10, 16, 24, 40, 51, 61],
      [12, 12, 14, 19, 26, 28, 60, 55],
      [14, 13, 16, 24, 40, 57, 69, 56],
      [14, 17, 22, 29, 51, 87, 80, 62],
      [18, 22, 37, 56, 68, 109, 103, 77],
      [24, 35, 55, 64, 81, 104, 113, 92],
      [49, 64, 78, 87, 103, 121, 120, 101],
      [72, 92, 95, 98, 112, 100, 103, 99]
]

(T_Y, T_cb, T_cr) = quantization(Y_dct, Cb_dct, Cr_dct, Z)
T_cr.shape, T_cr[0, :]


((17496, 8, 8),
 array([[-176.,   -0.,    0.,   -0.,    0.,   -0.,   -0.,   -0.],
        [-234.,    1.,    0.,   -0.,   -0.,   -0.,   -0.,   -0.],
        [-201.,   -0.,   -0.,   -0.,   -0.,   -0.,    0.,    0.],
        [-201.,    0.,    0.,   -0.,    0.,   -0.,    0.,    0.],
        [-156.,   -0.,   -0.,    0.,    0.,   -0.,    0.,    0.],
        [-117.,    0.,   -0.,   -0.,   -0.,   -0.,   -0.,   -0.],
        [ -58.,   -0.,   -0.,    0.,   -0.,   -0.,   -0.,   -0.],
        [ -39.,    0.,    0.,   -0.,   -0.,    0.,   -0.,    0.]]))

Step 6. Symbols encoding:
6.1  Zig-zag scanning


In [0]:
# https://rosettacode.org/wiki/Zig-zag_matrix#Python
import operator

def zigzag(n):
    '''zigzag rows'''
    def compare(xy):
        x, y = xy
        return (x + y, -y if (x + y) % 2 else y)
      
    xs = range(n)
    
    return {index: n for n, index in enumerate(sorted(
        ((x, y) for x in xs for y in xs),
        key=compare
    ))}


x = zigzag(8)
zig_zag_indices = sorted(x.items(), key=operator.itemgetter(1))
zig_zag_indices = [ i[0] for i in zig_zag_indices]

In [0]:
# zig zag representation
Y_zigzag = []
cb_zigzag = []
cr_zigzag = []
for block in range(T_Y.shape[0]):
  for i in zig_zag_indices:
    Y_zigzag.append(T_Y[block][i])
    cb_zigzag.append(T_cb[block][i])
    cr_zigzag.append(T_cr[block][i])
# Y_zigzag = [T_Y[block][i] for i in zig_zag_indices for block in range(T_Y.shape[0])]
# Cb_zigzag =  [[T_cb[block][i]for i in zig_zag_indices] for block in range(T_cb.shape[0])]
# Cr_zigzag =  [[T_cr[block][i]for i in zig_zag_indices] for block in range(T_cr.shape[0])]

# TEST 
# Y_zigzag[:3], [T_Y[0][0,0],T_Y[0][0,1],T_Y[0][1,0]], T_Y[0]
assert Y_zigzag[:3][0] == T_Y[0][0,0]
assert Y_zigzag[:3][1] == T_Y[0][0,1]
assert Y_zigzag[:3][2] == T_Y[0][1,0]
Y_zigzag = np.asarray(Y_zigzag)
cb_zigzag = np.asarray(cb_zigzag)
cr_zigzag = np.asarray(cr_zigzag)


In [0]:
# merge blocks
Y_merged = Y_zigzag.reshape((max_M, max_N))
Cb_merged = cb_zigzag.reshape((max_M, max_N))
Cr_merged = cr_zigzag.reshape((max_M, max_N))

6.2 Huffman coding

In [0]:
# count frequency of each color 
def count_color_freq(img):
  unique, counts = np.unique(img, return_counts=True)
  return dict(zip(unique, counts))

frequencies = [count_color_freq(chanel) for chanel in [Y_merged, Cb_merged, Cr_merged] ] 

total_pixels = max_N *max_M
assert max_N *max_M == sum(frequencies[0].values())

# frequencies = [ { k:v/total_pixels for k,v in f.items()} for f in frequencies]
probs = []
for f_index in range(len(frequencies)):
  probs.append({ k:v/total_pixels for k,v in frequencies[f_index].items()})
# probs[2]

In [0]:
# # !pip install dahuffman
# from dahuffman import HuffmanCodec

# codec = HuffmanCodec.from_frequencies({'e': 100, 'n':20, 'x':1, 'i': 40, 'q':3})
# encoded = codec.encode('exeneeeexniqneieini')
# codec.decode(encoded)

In [0]:
# https://gist.github.com/mreid/fdf6353ec39d050e972b
# Example Huffman coding implementation
# Distributions are represented as dictionaries of { 'symbol': probability }
# Codes are dictionaries too: { 'symbol': 'codeword' }

def huffman(p):
    '''Return a Huffman code for an ensemble with distribution p.'''
    assert(sum(p.values()) > 0.99) # Ensure probabilities sum to 1
    assert(sum(p.values()) < 1.01) # Ensure probabilities sum to 1

    # Base case of only two symbols, assign 0 or 1 arbitrarily
    if(len(p) == 2):
        return dict(zip(p.keys(), ['0', '1']))

    # Create a new distribution by merging lowest prob. pair
    p_prime = p.copy()
    a1, a2 = lowest_prob_pair(p)
    p1, p2 = p_prime.pop(a1), p_prime.pop(a2)
    p_prime[a1 + a2] = p1 + p2

    # Recurse and construct code on new distribution
    c = huffman(p_prime)
    ca1a2 = c.pop(a1 + a2)
    c[a1], c[a2] = ca1a2 + '0', ca1a2 + '1'

    return c

def lowest_prob_pair(p):
    '''Return pair of symbols from distribution p with lowest probabilities.'''
    assert(len(p) >= 2) # Ensure there are at least 2 symbols in the dist.

    sorted_p = sorted(p.items())
    return sorted_p[0][0], sorted_p[1][0]

# Example execution
# ex1 = { 'a': 0.5, 'b': 0.25, 'c': 0.25 }
# huffman(ex1) # => {'a': '0', 'c': '10', 'b': '11'}
# huffmans = [huffman(f) for f in range(len(frequencies))]



huffman_trees = [ huffman(probs[i]) for i in range(3)]

# probs[0]

In [0]:
huffman_trees[0]

b) Take a colour image test.png. Compress this image by your JPEG algorithm implemen-
tation.

(c) Perform the image reconstruction.

(d) Display the original and reconstructed images. Compare those images based on the MSE.
Make a conclusion about the compression efficiency based on the visual images quality
and the files size

In [0]:

def inverse_extract_colors(R, G, B):
  img = np.zeros(M, N, 3)
  img[:, :, 0] = R
  img[:, :, 1] = G
  B = img[:, :, 2] = B
  return img


In [0]:



def YCbCr( R_DIMENSION, G_DIMENSION, B_DIMENSION): 
  
  Y = np.add(np.add(0.299 * R_DIMENSION, 0.587 * G_DIMENSION), 0.114 * B_DIMENSION)
  
  Cb = - 0.1687 * R_DIMENSION - 0.3313 * G_DIMENSION + 0.5 * B_DIMENSION + 128
  
  Cr = 0.5 * R_DIMENSION - 0.4187 * G_DIMENSION - 0.0813 * B_DIMENSION + 128
  
  R_DIMENSION = Y + 1.402 * (Cr - 128)
  
  G_DIMENSION = Y - 0.34414 * (Cb - 128) - 0.71414 * (Cr - 128)
  
  B_DIMENSION = Y + 1.772 * (Cb - 128)
  
  return Y, Cb, Cr, R_DIMENSION, G_DIMENSION, B_DIMENSION




In [0]:
# upsample
# https://stackoverflow.com/questions/13242382/resampling-a-numpy-array-representing-an-image
def upsample():
  print 'Resampled by a factor of 2 with nearest interpolation:'
  print scipy.ndimage.zoom(x, 2, order=0)


  print 'Resampled by a factor of 2 with bilinear interpolation:'
  print scipy.ndimage.zoom(x, 2, order=1)

In [0]:
# blck 2 images 
def img_2_block(Y, cr, max_M, max_N):
  number_octads = int(max_M* max_N / 64)
  Y_r = Y.reshape((number_octads, 8,8))
  Cb_r = Cb.reshape((number_octads, 8,8))
  Cr_r = Cr.reshape((number_octads, 8,8))
  return Y_r, Cb_r,Cr_r


def block_2_img(Y_r, Cb_r,Cr_r, max_M, max_N):
  Y = Y_r.reshape((max_M, max_N)
  Cb = Cb_r.reshape((max_M, max_N))
  Cr = Cr_r.reshape((max_M, max_N))
  return Y, cb, cr